## CC (Elia)

In [37]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer

In [38]:
# Load data
output_folder = Path('../output')
X = np.load(output_folder / 'x_remb_CC.npy')
y_df = pd.read_csv(output_folder / 'y_df_CC.csv')
# X_sparse = csr_matrix(X)

In [39]:
import ast

y_df['GO_term'] = y_df['GO_term'].apply(lambda x: list(ast.literal_eval(x)))

In [40]:
# Preprocess target (MultiLabelBinarizer for multi-label classification)
mlb = MultiLabelBinarizer()  # Enable sparse output
y = mlb.fit_transform(y_df['GO_term'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [41]:
print("X_sparse shape:", X.shape)
print("y shape:", y.shape)

X_sparse shape: (84638, 274)
y shape: (84638, 678)


In [42]:
X = pd.DataFrame(X.tolist()) 

In [43]:
y_new = y.astype(np.float32)
y_new

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

device = torch.device("cpu")

In [48]:
# Convert data to PyTorch tensors
# Convert data to PyTorch tensors (using full embeddings)
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32).to(device)  # Full embeddings: shape (44558, 1024)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)  # Labels: shape (44558, num_labels)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)    # Full embeddings: shape (test_size, 1024)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)  # Labels: shape (test_size, num_labels)


# Define the Neural Network
class MultilabelNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MultilabelNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Dropout(0.3),  # Dropout for regularization
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_sizes[1], output_size),
            nn.Sigmoid()  # For multilabel classification
        )
    
    def forward(self, x):
        return self.model(x)

# Define model parameters
input_size = X_train_tensor.shape[1]  # 300 features
hidden_sizes = [1024, 512, 256]            # Hidden layer sizes
output_size = y_train.shape[1]    # Number of labels

# Initialize the model
model = MultilabelNN(input_size, hidden_sizes, output_size).to(device)

# Loss function and optimizer

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multilabel
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
batch_size = 1024

# Function to train the model
def train_model(model, X_train, y_train, optimizer, criterion, batch_size, epochs):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

# Train the model
train_model(model, X_train_tensor, y_train_tensor, optimizer, criterion, batch_size, epochs)

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_binary = (y_pred.cpu().numpy() > 0.5).astype(int)

# Classification report
target_names = [str(cls) for cls in mlb.classes_]
print("\nNeural Network Results:")
print(classification_report(y_test, y_pred_binary, target_names=target_names))

Epoch [1/50], Loss: 6.0130
Epoch [2/50], Loss: 2.9623
Epoch [3/50], Loss: 2.9182
Epoch [4/50], Loss: 2.8930
Epoch [5/50], Loss: 2.8750
Epoch [6/50], Loss: 2.8602
Epoch [7/50], Loss: 2.8497
Epoch [8/50], Loss: 2.8394
Epoch [9/50], Loss: 2.8318
Epoch [10/50], Loss: 2.8260


KeyboardInterrupt: 

In [19]:
import torch 
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, f1_score, precision_recall_fscore_support
from sklearn.model_selection import RepeatedKFold
import numpy as np
import time  # Import time module


# Define the PyTorch model with larger layers
class MultiLabelNN(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(MultiLabelNN, self).__init__()
        self.model=nn.Sequential(nn.Linear(n_inputs, 2048),  # Very large hidden layer
                                 nn.ReLU(),
                                 nn.Dropout(0.3),
                                 nn.Linear(2048, 1024),
                                 nn.ReLU(),
                                 nn.Dropout(0.3),
                                 nn.Linear(1024, n_outputs),
                                 nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Train and evaluate the model
def train_and_evaluate_model(X, y, n_splits=3, n_repeats=1, epochs=30, batch_size=32, threshold=0.5):
    results = {"macro_f1": [], "micro_f1": [], "weighted_f1": [], "sample_f1": []}
    precision_recall_summary = []

    n_inputs, n_outputs = X.shape[1], y.shape[1]
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1)

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X), 1):
        # Split data
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

        # Initialize model, loss, and optimizer
        model = MultiLabelNN(n_inputs, n_outputs)
        
        criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        #optimizer = optim.Adam(model.parameters(), lr=1e-4)

        # Record start time for the entire fold
        start_time = time.time()

        print(f"FOLD {fold_idx}")

        # Training loop
        for epoch in range(epochs):
            epoch_start_time = time.time()  # Start time for the current epoch
            model.train()
            optimizer.zero_grad()

            # Forward pass
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Calculate elapsed time for the current epoch
            epoch_elapsed_time = time.time() - epoch_start_time
            avg_epoch_time = epoch_elapsed_time

            # Estimate remaining time
            remaining_epochs = epochs - (epoch + 1)
            remaining_time = remaining_epochs * avg_epoch_time

            # Print progress
            if (epoch % 10 == 0):
                print(f'Epoch {epoch + 1}/{epochs}\n'
                    f'Epoch Time: {epoch_elapsed_time:.1f}s, '
                    f'Estimated Remaining Time: {remaining_time:.0f}s')
            elif (epoch == epochs - 1):
                print(f'Epoch {epochs}/{epochs}\n'
                    f'Epoch Time: {epoch_elapsed_time:.1f}s')

        # Evaluation
        model.eval()
        with torch.no_grad():
            yhat = model(X_test_tensor).numpy()
            yhat_binary = (yhat > threshold).astype(int)  # Convert probabilities to binary predictions

            # Calculate metrics
            metrics = precision_recall_fscore_support(y_test, yhat_binary, average=None, zero_division=0)
            precision, recall, f1_scores, _ = metrics
            macro_f1 = f1_score(y_test, yhat_binary, average='macro')
            micro_f1 = f1_score(y_test, yhat_binary, average='micro')
            weighted_f1 = f1_score(y_test, yhat_binary, average='weighted')
            sample_f1 = f1_score(y_test, yhat_binary, average='samples')

            # Store results
            results["macro_f1"].append(macro_f1)
            results["micro_f1"].append(micro_f1)
            results["weighted_f1"].append(weighted_f1)
            results["sample_f1"].append(sample_f1)

            precision_recall_summary.append(
                {"precision_avg": np.mean(precision), "recall_avg": np.mean(recall)}
            )

            print(f'Fold {fold_idx}: Macro F1: {macro_f1:.3f}, Micro F1: {micro_f1:.3f}, '
                  f'Weighted F1: {weighted_f1:.3f}, Sample F1: {sample_f1:.3f}')

        # Calculate total time for the fold
        total_fold_time = time.time() - start_time
        print(f'Total time for fold {fold_idx}: {total_fold_time:.2f}s\n')

    return results, precision_recall_summary


# Run the model
results, precision_recall_summary = train_and_evaluate_model(X_new, y_new)


# Summarize performance
for key in results:
    print(f'{key}: Mean={np.mean(results[key]):.3f}, Std={np.std(results[key]):.3f}')

print('Average Precision/Recall per fold:')
for i, summary in enumerate(precision_recall_summary, 1):
    print(f'Fold {i}: Precision: {summary["precision_avg"]:.3f}, Recall: {summary["recall_avg"]:.3f}')

FOLD 1
Epoch 1/30
Epoch Time: 5.9s, Estimated Remaining Time: 171s
Epoch 11/30
Epoch Time: 4.4s, Estimated Remaining Time: 84s
Epoch 21/30
Epoch Time: 4.4s, Estimated Remaining Time: 40s
Epoch 30/30
Epoch Time: 4.4s
Fold 1: Macro F1: 0.010, Micro F1: 0.563, Weighted F1: 0.392, Sample F1: 0.579
Total time for fold 1: 149.43s

FOLD 2
Epoch 1/30
Epoch Time: 5.2s, Estimated Remaining Time: 151s
Epoch 11/30
Epoch Time: 5.5s, Estimated Remaining Time: 104s
Epoch 21/30
Epoch Time: 6.0s, Estimated Remaining Time: 54s


KeyboardInterrupt: 

# Noccapito

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Convert sparse matrices to dense arrays
X_array = X_sparse.toarray()
Y_array = y.toarray()

X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.2)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

## RF

In [26]:
# Handle class imbalance: Compute class weights for multilabel classification
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Create a RandomForestClassifier
rf_clf = RandomForestClassifier(
    n_estimators=10,       # Number of trees
    max_depth=4,          # Maximum depth of trees
    n_jobs=-1,             # Use all CPU cores
    random_state=42,
    verbose=5,
    class_weight=class_weight_dict  # Use the computed class weights
)

# Create the MultiOutputClassifier
multi_clf = MultiOutputClassifier(rf_clf, n_jobs=-1)

# Fit the model
multi_clf.fit(X_train_scaled, y_train)

# Evaluate on the test set
y_pred = multi_clf.predict(X_test_scaled)

KeyboardInterrupt: 

In [21]:
report = classification_report(y_test, y_pred, target_names=mlb.classes_)
print(report)

C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

  GO:0000118       0.00      0.00      0.00        44
  GO:0000123       0.00      0.00      0.00        67
  GO:0000124       0.00      0.00      0.00        23
  GO:0000131       0.00      0.00      0.00        15
  GO:0000137       0.00      0.00      0.00        20
  GO:0000138       0.00      0.00      0.00        25
  GO:0000139       0.00      0.00      0.00       133
  GO:0000145       0.00      0.00      0.00         8
  GO:0000151       0.00      0.00      0.00       131
  GO:0000152       0.00      0.00      0.00        30
  GO:0000176       0.00      0.00      0.00         5
  GO:0000178       0.00      0.00      0.00        11
  GO:0000228       0.00      0.00      0.00       159
  GO:0000235       0.00      0.00      0.00        15
  GO:0000307       0.00      0.00      0.00        26
  GO:0000313       0.00      0.00      0.00        85
  GO:0000314       0.00      0.00      0.00        31
  GO:0000315       0.00    

## NN

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Define the model
model = Sequential()
model.add(Input(shape=(X_train_scaled.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=[Precision(), Recall(), AUC(name='auc')])

# Train the model
history = model.fit(X_train_scaled, y_train,
                    epochs=100,
                    batch_size=128,
                    validation_data=(X_test_scaled, y_test),
                    class_weight=class_weight_dict)

Epoch 1/100
529/529 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - auc: 0.8176 - loss: 0.1829 - precision_6: 0.2209 - recall_6: 0.4351 - val_auc: 0.9151 - val_loss: 0.0510 - val_precision_6: 0.7431 - val_recall_6: 0.4542
Epoch 2/100
529/529 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - auc: 0.9072 - loss: 0.0616 - precision_6: 0.7438 - recall_6: 0.4499 - val_auc: 0.9206 - val_loss: 0.0506 - val_precision_6: 0.7431 - val_recall_6: 0.4542
Epoch 3/100
529/529 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - auc: 0.9099 - loss: 0.0614 - precision_6: 0.7432 - recall_6: 0.4500 - val_auc: 0.9206 - val_loss: 0.0502 - val_precision_6: 0.7431 - val_recall_6: 0.4542
Epoch 4/100
529/529 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - auc: 0.9117 - loss: 0.0606 - precision_6: 0.7420 - recall_6: 0.4512 - val_auc: 0.9226 - val_loss: 0.0500 - val_precision_6: 0.7431 - val_recall_6: 0.4542
Epoch 5/100
529/529 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - auc: 0.9122 - loss: 0.0594 - precision_6: 0.7430 - recall_6: 0.4487 - val_auc: 0.9228 - val_loss: 0.0506 - va

In [50]:
report = classification_report(y_test, model.predict(X_test_scaled) > 0.3, target_names=mlb.classes_)
print(report)

529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  
              precision    recall  f1-score   support

  GO:0000118       0.00      0.00      0.00        44
  GO:0000123       0.00      0.00      0.00        67
  GO:0000124       0.00      0.00      0.00        23
  GO:0000131       0.00      0.00      0.00        15
  GO:0000137       0.00      0.00      0.00        20
  GO:0000138       0.00      0.00      0.00        25
  GO:0000139       0.00      0.00      0.00       133
  GO:0000145       0.00      0.00      0.00         8
  GO:0000151       0.00      0.00      0.00       131
  GO:0000152       0.00      0.00      0.00        30
  GO:0000176       0.00      0.00      0.00         5
  GO:0000178       0.00      0.00      0.00        11
  GO:0000228       0.00      0.00      0.00       159
  GO:0000235       0.00      0.00      0.00        15
  GO:0000307       0.00      0.00      0.00        26
  GO:0000313       0.00      0.00      0.00        85
  GO:0000314       0.00      0.00     

C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## New NN

In [14]:
from numpy import mean, std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import f1_score
import numpy as np

# Define the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# Evaluate the model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results_f1 = []
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    n_splits, n_repeats = 5, 3
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1)
    estimate = n_splits * n_repeats
    i = 0
    
    for train_ix, test_ix in cv.split(X):
        i += 1
        # Prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # Define model
        model = get_model(n_inputs, n_outputs)
        # Fit model
        model.fit(X_train, y_train, verbose=5, epochs=10)
        # Make a prediction
        yhat = model.predict(X_test)
        
        # Ensure probabilities are within (0, 1.000]
        yhat = np.clip(yhat, 1e-6, 1.0)
        
        # Calculate F1-score (macro)
        f1 = f1_score(y_test, (yhat > 0.5).astype(int), average='macro')
        print(f'{i}/{estimate} F1-Score {f1:.3f}')
        results_f1.append(f1)
    
    return results_f1

# Load your data (X_array, Y_array)
# X_array: Features, Y_array: Multi-label targets
# Example: X_array.shape = (n_samples, n_features), Y_array.shape = (n_samples, n_classes)

results_f1 = evaluate_model(X_array, Y_array)

# Summarize performance
print('F1-Score: %.3f (%.3f)' % (mean(results_f1), std(results_f1)))

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/15 F1-Score 0.010
Epoch 1/10


C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
2/15 F1-Score 0.010
Epoch 1/10


C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
3/15 F1-Score 0.010


...

C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
12/15 F1-Score 0.010


C:\Users\eliat\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


KeyboardInterrupt: 